In [ ]:
import socket
import json
import logging
import time
from web3 import Web3
import os
from eth_account.messages import encode_defunct
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

# Set working directory
project_path = "D:/blockchain project"
if os.path.exists(project_path):
    os.chdir(project_path)
else:
    print(f"❌ Project path not found: {project_path}")
    exit(1)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Connect to Ganache
ganache_url = "http://127.0.0.1:8545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if not web3.is_connected():
    logger.error("❌ Failed to connect to Ganache. Make sure it's running.")
    exit(1)

# Load deployed smart contract
try:
    with open("build/contracts/AuthSignatureManager.json", "r") as f:
        contract_json = json.load(f)
        abi = contract_json.get("abi")
        if not abi:
            raise KeyError("ABI key is missing in JSON file")
except FileNotFoundError:
    logger.error("❌ Contract JSON file not found! Compile and deploy the contract first.")
    exit(1)
except KeyError as e:
    logger.error(f"❌ Error: {e}")
    exit(1)
except Exception as e:
    logger.error(f"❌ Unexpected error: {e}")
    exit(1)

# Update this with your deployed contract address
contract_address = "0xD8B774A2273859AE7f7470cfe26cBCea985389DD"
contract = web3.eth.contract(address=contract_address, abi=abi)

# Verify contract deployment
contract_code = web3.eth.get_code(contract_address)
if contract_code == "0x":
    logger.error("❌ Contract not deployed at the specified address.")
    exit(1)

# Socket setup
host = '127.0.0.1'
port = 55555

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
from_address = web3.eth.accounts[0]  # Aligned with server

ENCRYPTION_KEY = None  # Will be set dynamically after authorization

def decrypt_message(encrypted_message):
    iv = encrypted_message[:16]
    cipher = AES.new(ENCRYPTION_KEY, AES.MODE_CBC, iv=iv)
    decrypted_padded = cipher.decrypt(encrypted_message[16:])
    return unpad(decrypted_padded, AES.block_size).decode()

def get_message_hash(sender, message):
    start_time = time.time()
    sender = Web3.to_checksum_address(sender)
    hash_result = Web3.solidity_keccak(["address", "string"], [sender, message])
    end_time = time.time()
    logger.info(f"⏱ Hash generation time: {(end_time - start_time)*1000:.2f} ms")
    return hash_result

def verify_signature(message, signature, expected_address):
    start_time = time.time()
    message_hash = get_message_hash(expected_address, message)
    prefixed_message = encode_defunct(hexstr=message_hash.hex())
    recovered = web3.eth.account.recover_message(prefixed_message, signature=signature)
    end_time = time.time()
    logger.info(f"⏱ Signature verification time: {(end_time - start_time)*1000:.2f} ms")
    return recovered == expected_address

def connect_to_server():
    start_time = time.time()
    try:
        client_socket.connect((host, port))
        end_time = time.time()
        logger.info(f"⏱ Connection time: {(end_time - start_time)*1000:.2f} ms")
        logger.info(f"🔗 Connected to server at {host}:{port}")
        return True
    except Exception as e:
        logger.error(f"❌ Connection failed: {e}")
        return False

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    if not connect_to_server():
        exit(1)

    auth_message = "Authorize me"
    client_socket.sendall(auth_message.encode())
    response = client_socket.recv(1024)
    
    if response.startswith(b"AUTHORIZED:"):
        global ENCRYPTION_KEY
        ENCRYPTION_KEY = response[len(b"AUTHORIZED:"):len(b"AUTHORIZED:") + 16]
        logger.info(f"🔑 Received encryption key: {ENCRYPTION_KEY.hex()}")
        logger.info("✅ Successfully authorized")
    else:
        logger.error(f"❌ Authorization failed: {response.decode()}")
        exit(1)

    last_signature = None
    last_message = None

    while True:
        message = input("Enter message (type 'exit' to quit, 'verify' to check last signature): ").strip()
        if message.lower() == "exit":
            break
        elif message.lower() == "verify" and last_signature and last_message:
            decrypted_message = decrypt_message(last_message)
            is_valid = verify_signature(decrypted_message, last_signature, from_address)
            logger.info(f"Signature verification: {'✅ Valid' if is_valid else '❌ Invalid'}")
            continue

        start_time = time.time()
        client_socket.sendall(message.encode())
        response = client_socket.recv(2048)
        end_time = time.time()
        logger.info(f"⏱ Server response time: {(end_time - start_time)*1000:.2f} ms")

        if response.startswith(b"ERROR"):
            logger.error(f"❌ Server error: {response.decode()}")
            continue
        elif response.startswith(b"REAUTH"):
            logger.info(f"⏰ {response.decode()}")
            choice = input("Enter 'yes' to reauthorize, 'no' to disconnect: ").strip().lower()
            client_socket.sendall(choice.encode())
            reauth_response = client_socket.recv(1024)
            if reauth_response.startswith(b"AUTHORIZED:"):
                ENCRYPTION_KEY = reauth_response[len(b"AUTHORIZED:"):len(b"AUTHORIZED:") + 16]
                logger.info(f"🔑 Updated encryption key: {ENCRYPTION_KEY.hex()}")
                logger.info("✅ Reauthorized successfully")
            elif reauth_response.startswith(b"DISCONNECT"):
                logger.info("❌ Connection terminated by server")
                break
            else:
                logger.error(f"❌ Reauthorization failed: {reauth_response.decode()}")
                break
        else:
            signature_length = int.from_bytes(response[:4], 'big')
            signature = response[4:4 + signature_length]
            encrypted_message = response[4 + signature_length:]
            logger.info(f"🔒 Received encrypted message: {encrypted_message.hex()}")

            decrypted_message = decrypt_message(encrypted_message)
            logger.info(f"📩 Decrypted message for verification: {decrypted_message}")
            is_valid = verify_signature(decrypted_message, signature, from_address)
            logger.info(f"Signature verification: {'✅ Valid' if is_valid else '❌ Invalid'}")
            
            last_signature = signature
            last_message = encrypted_message

finally:
    client_socket.close()
    logger.info("🔌 Connection closed")
            

2025-03-10 18:27:57,108 - INFO - ⏱ Connection time: 0.27 ms
2025-03-10 18:27:57,110 - INFO - 🔗 Connected to server at 127.0.0.1:55555
2025-03-10 18:27:57,319 - INFO - 🔑 Received encryption key: 2c960e16a4bddb4c2596bdb3db3042ee
2025-03-10 18:27:57,320 - INFO - ✅ Successfully authorized


Enter message (type 'exit' to quit, 'verify' to check last signature):  Implementation of Zero Trust Architecture with Blockchain for Secure Communication


2025-03-10 18:28:05,682 - INFO - ⏱ Server response time: 236.85 ms
2025-03-10 18:28:05,683 - INFO - 🔒 Received encrypted message: 12cfdd2845a078dcf39495a0ce6f22d746a0ff60ae51c0236fe5c685173a3132f29ee11a7f4705ec5e92700b474832e3bed619cacd736ac244204c3d7b12da0ce2ebee3f83817ddc700aa05a194873456ae20be372c50e7d96012e764de5cf76a25a8ec518ad8e4c0b27de70e6ccb482
2025-03-10 18:28:05,684 - INFO - 📩 Decrypted message for verification: Implementation of Zero Trust Architecture with Blockchain for Secure Communication
2025-03-10 18:28:05,696 - INFO - ⏱ Hash generation time: 11.04 ms
2025-03-10 18:28:05,721 - INFO - ⏱ Signature verification time: 35.60 ms
2025-03-10 18:28:05,723 - INFO - Signature verification: ✅ Valid


Enter message (type 'exit' to quit, 'verify' to check last signature):  connect to the server


2025-03-10 18:41:26,838 - INFO - ⏱ Server response time: 216.81 ms
2025-03-10 18:41:26,839 - ERROR - ❌ Server error: ERROR: Transaction failed


Enter message (type 'exit' to quit, 'verify' to check last signature):  try to connect to the server


2025-03-10 18:41:46,656 - INFO - ⏱ Server response time: 47.64 ms
2025-03-10 18:41:46,658 - INFO - ⏰ REAUTH: Do you want to reauthorize? (yes/no)
